In [9]:
import os, glob, urllib.parse
import pandas as pd
from sqlalchemy import create_engine, event, text

# ========= PARAMÈTRES SQL =========
SERVER   = "LAPTOP-V857ADI3"            # machine locale
DATABASE = "olist_db_staging"
USER     = "userairflow"
PWD      = "air123"
DRIVER   = "ODBC Driver 17 for SQL Server"   # tu as le 17 installé
SCHEMA   = "staging"                         # schéma cible

# ========= DOSSIER CSV =========
CSV_DIR = r"C:\Users\saleban ali hassan\Documents\olist"

# ========= MAPPING nom_fichier -> nom_table =========
NAME_MAP = {
    "olist_customers_dataset": "customers",
    "olist_geolocation_dataset": "geolocation",
    "olist_order_items_dataset": "order_items",
    "olist_order_payments_dataset": "order_payments",
    "olist_order_reviews_dataset": "order_reviews",
    "olist_orders_dataset": "orders",
    "olist_products_dataset": "products",
    "olist_sellers_dataset": "sellers",
    "product_category_name_translation": "product_category_name_translation",
}

# ========= CONNEXION (SQLAlchemy + pyodbc) =========
odbc_str = (
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USER};PWD={PWD};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
params = urllib.parse.quote_plus(odbc_str)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Accélère executemany côté pyodbc
@event.listens_for(engine, "before_cursor_execute")
def _fast_exec(conn, cursor, statement, parameters, context, executemany):
    if executemany:
        try:
            cursor.fast_executemany = True
        except Exception:
            pass

# ========= Créer le schéma 'staging' s'il n'existe pas =========
with engine.begin() as conn:
    exists = conn.execute(
        text("SELECT 1 FROM sys.schemas WHERE name = :n"), {"n": SCHEMA}
    ).first()
    if not exists:
        conn.exec_driver_sql(f"CREATE SCHEMA [{SCHEMA}]")
    # test rapide
    conn.execute(text("SELECT 1"))

# ========= Fonctions utils =========
def table_name_from_path(path: str) -> str:
    stem = os.path.splitext(os.path.basename(path))[0]
    stem = stem.strip().lower().replace("-", "_").replace(" ", "_")
    return NAME_MAP.get(stem, stem)  # fallback = nom normalisé

# ========= Import CSV -> staging.<table> =========
csv_files = sorted(glob.glob(os.path.join(CSV_DIR, "*.csv")))
if not csv_files:
    raise FileNotFoundError(f"Aucun CSV trouvé dans {CSV_DIR}")

for csv_path in csv_files:
    table = table_name_from_path(csv_path)
    print(f"--> {os.path.basename(csv_path)}  -> [{SCHEMA}.{table}]")

    df = pd.read_csv(csv_path, low_memory=False)
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # Limite SQL Server: 2100 paramètres par requête
    # On choisit un chunksize sûr: floor(2100 / nb_colonnes)
    ncols = max(len(df.columns), 1)
    safe_chunk = max(1, min(10000, 2100 // ncols))

    df.to_sql(
        name=table,
        con=engine,
        schema=SCHEMA,
        if_exists="replace",     # "append" pour ajouter
        index=False,
        chunksize=safe_chunk,    # OK avec fast_executemany
        method=None,             # pas de multi-VALUES => évite la limite 2100
    )
    print(f"    OK ({len(df):,} lignes, chunksize={safe_chunk})")

print("✅ Import terminé dans le schéma 'staging' de la base olist_staging.")


--> olist_customers_dataset.csv  -> [staging.customers]
    OK (99,441 lignes, chunksize=420)
--> olist_geolocation_dataset.csv  -> [staging.geolocation]
    OK (1,000,163 lignes, chunksize=420)
--> olist_order_items_dataset.csv  -> [staging.order_items]
    OK (112,650 lignes, chunksize=300)
--> olist_order_payments_dataset.csv  -> [staging.order_payments]
    OK (103,886 lignes, chunksize=420)
--> olist_order_reviews_dataset.csv  -> [staging.order_reviews]
    OK (99,224 lignes, chunksize=300)
--> olist_orders_dataset.csv  -> [staging.orders]
    OK (99,441 lignes, chunksize=262)
--> olist_products_dataset.csv  -> [staging.products]
    OK (32,951 lignes, chunksize=233)
--> olist_sellers_dataset.csv  -> [staging.sellers]
    OK (3,095 lignes, chunksize=525)
--> product_category_name_translation.csv  -> [staging.product_category_name_translation]
    OK (71 lignes, chunksize=1050)
✅ Import terminé dans le schéma 'staging' de la base olist_staging.
